In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName('QandA Airbnb data - aggregate and joining') \
    .getOrCreate()

In [2]:
london_listings = spark.read.csv(
    path='data/airbnb-london-listings.csv.gz',
    header=True,
    inferSchema=True,
    sep=',',
    quote='"',
    escape='"',
    multiLine=True,
    mode='PERMISSIVE'
)

london_reviews = spark.read.csv(
    path='data/airbnb-london-reviews.csv.gz',
    header=True,
    inferSchema=True,
    sep=',',
    quote='"',
    escape='"',
    multiLine=True,
    mode='PERMISSIVE'
)

In [3]:
london_listings.printSchema()

root
 |-- id: long (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: date (nullable = true)
 |-- source: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: date (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_count: int

In [4]:
london_reviews.printSchema()

root
 |-- listing_id: long (nullable = true)
 |-- id: long (nullable = true)
 |-- date: date (nullable = true)
 |-- reviewer_id: integer (nullable = true)
 |-- reviewer_name: string (nullable = true)
 |-- comments: string (nullable = true)



In [20]:
# 1. Count the number of reviews per listing using the "reviews" dataset

london_reviews \
    .groupby(london_reviews.listing_id) \
    .count() \
    .orderBy('count', ascending=[False]) \
    .show(10)

+----------+-----+
|listing_id|count|
+----------+-----+
|  47408549| 1902|
|  43120947| 1647|
|  19670926| 1443|
|   2126708| 1142|
|  46233904| 1002|
|   2659707|  998|
|  27833488|  951|
|   4748665|  933|
|  42081759|  914|
|   5266466|  909|
+----------+-----+
only showing top 10 rows



In [38]:
# 2. Compute the total number of listings and average review score per host
from pyspark.sql.functions import avg, count
london_listings \
    .filter(london_listings.review_scores_rating.isNotNull()) \
    .groupby(london_listings.host_id) \
    .agg(
        avg(london_listings.review_scores_rating).alias('avg_review_score'),
        count(london_listings.id).alias('total_listings')
    ) \
    .orderBy('total_listings', 'avg_review_score', ascending=[False, False]) \
    .show(10)

+---------+------------------+--------------+
|  host_id|  avg_review_score|total_listings|
+---------+------------------+--------------+
| 28820321| 4.589805447470818|           257|
|  1432477|4.3452765957446795|           235|
|314162972|  4.35020202020202|           198|
| 33889201| 4.527708333333332|           144|
|228928499| 4.799421487603304|           121|
|156158778| 4.892184873949581|           119|
|124359784| 4.434789915966386|           119|
| 83740964| 4.431452991452991|           117|
| 30253178| 4.436851851851852|           108|
| 89355192|4.6719191919191925|            99|
+---------+------------------+--------------+
only showing top 10 rows



In [50]:
# 3: Find the top ten listings with the highest number of reviews

london_reviews \
    .groupby(london_reviews.listing_id) \
    .count() \
    .orderBy('count', ascending=[False]) \
    .limit(10) \
    .show()

+----------+-----+
|listing_id|count|
+----------+-----+
|  47408549| 1902|
|  43120947| 1647|
|  19670926| 1443|
|   2126708| 1142|
|  46233904| 1002|
|   2659707|  998|
|  27833488|  951|
|   4748665|  933|
|  42081759|  914|
|   5266466|  909|
+----------+-----+



In [54]:
# 4. Find the top five neighborhoods with the most listings

london_listings \
    .groupby(london_listings.neighbourhood_cleansed) \
    .count() \
    .orderBy('count', ascending=False) \
    .limit(5) \
    .show(truncate=False)

+----------------------+-----+
|neighbourhood_cleansed|count|
+----------------------+-----+
|Westminster           |11385|
|Tower Hamlets         |7469 |
|Camden                |6551 |
|Kensington and Chelsea|6401 |
|Hackney               |6359 |
+----------------------+-----+



In [60]:
# 5. Get a data frame with the following four columns:
# * Listing's ID
# * Listing's name
# * Reviewer's name
# * Review's comment
# Use "join" to combine data from two datasets

london_listings_reviews = london_listings \
    .join(other=london_reviews, 
          on=london_listings.id == london_reviews.listing_id, 
          how='inner')

london_listings_reviews \
    .select(london_listings.id,
            london_listings.name,
            london_reviews.reviewer_name,
            london_reviews.comments) \
    .show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 id            | 13913                                                                                      

In [67]:
# 6.Get top five listings with the highest average review comment length. Only return listings with at least 5 reviews
# Use the "length" function from the "pyspark.sql.functions" to get a lenght of a review

from pyspark.sql.functions import length

london_reviews \
    .withColumn('comment_length', length(london_reviews.comments)) \
    .groupby(london_reviews.listing_id) \
    .agg(
        avg('comment_length').alias('avg_comment_length'),
        count(london_reviews.id).alias('reviews_count')
    ) \
    .filter('reviews_count >= 5') \
    .orderBy('avg_comment_length', ascending=False) \
    .show()

+------------------+------------------+-------------+
|        listing_id|avg_comment_length|reviews_count|
+------------------+------------------+-------------+
|618608352812465378|1300.1666666666667|            6|
|          28508447|1089.3333333333333|            6|
|          22661311| 1035.857142857143|            7|
|          53145228|1006.6666666666666|            6|
|627425975703032358| 951.7777777777778|            9|
|           2197681|             939.2|            5|
|          13891813|             905.0|            5|
|            979753| 893.9230769230769|           13|
|630150178279666225| 890.7272727272727|           11|
|           8856894| 890.1666666666666|            6|
|          33310686| 885.8333333333334|            6|
|          22524075|             885.0|            5|
|          29469389|             885.0|            6|
|           5555679| 878.7169811320755|          106|
|           6594477|             863.6|            5|
|          33385444|        

In [76]:
# 7. Using the "join" operator find listings without reviews.
# Hint: Use "left_join" or "left_anti" join type when implementing this

london_listings \
    .join(other=london_reviews, 
          on=london_listings.id == london_reviews.listing_id, 
          how='left_anti') \
    .select(london_listings.id,
            london_listings.name) \
    .orderBy('id') \
    .show(truncate=False)

+------+-------------------------------------------------+
|id    |name                                             |
+------+-------------------------------------------------+
|42692 |Fabulous flat w garden and bkfst!                |
|81951 |LONDON DETACHED HOUSE*ElecGates etc              |
|87827 |ChiqDoube Room in PrivateAppartment              |
|127860|Double bedroom in cottage Twickenham (sleeps 1-2)|
|208952|PENDA'S MEAD HOUSE                               |
|282299|Room in friendly house share                     |
|293275|Single Room in Creative House                    |
|296720|bedroom to rent in SW17 - London                 |
|300333|london is waiting for you                        |
|301631|Single Room for the Olympic Games!               |
|318986|Central London flat for Olympics                 |
|324055|worth to stay.you will never regret              |
|326146|Rent for Olympics                                |
|329610|ROOM TO RENT IN THE OLYMPIC PERIOD              

In [77]:
london_listings \
    .join(other=london_reviews, 
          on=london_listings.id == london_reviews.listing_id, 
          how='left_outer') \
    .filter(london_reviews.id.isNull()) \
    .select(london_listings.id,
            london_listings.name) \
    .orderBy('id') \
    .show(truncate=False)

+------+-------------------------------------------------+
|id    |name                                             |
+------+-------------------------------------------------+
|42692 |Fabulous flat w garden and bkfst!                |
|81951 |LONDON DETACHED HOUSE*ElecGates etc              |
|87827 |ChiqDoube Room in PrivateAppartment              |
|127860|Double bedroom in cottage Twickenham (sleeps 1-2)|
|208952|PENDA'S MEAD HOUSE                               |
|282299|Room in friendly house share                     |
|293275|Single Room in Creative House                    |
|296720|bedroom to rent in SW17 - London                 |
|300333|london is waiting for you                        |
|301631|Single Room for the Olympic Games!               |
|318986|Central London flat for Olympics                 |
|324055|worth to stay.you will never regret              |
|326146|Rent for Olympics                                |
|329610|ROOM TO RENT IN THE OLYMPIC PERIOD              

In [78]:
spark.stop()